In [1]:
import sys
import os
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('./test_score.ipynb'))))

In [2]:
import numpy as np
import pandas as pd

from ylearn.estimator_model._naive_forest import _GrfTree, NaiveGrf
from ylearn.estimator_model import CausalTree
from ylearn.exp_dataset.exp_data import sq_data
from ylearn.utils._common import to_df
from ylearn.estimator_model._naive_forest.utils import grad, grad_coef, inverse_grad

from numpy.linalg import lstsq

In [3]:
from copy import deepcopy


n = 2000
d = 10     
n_x = 1
y, x, v = sq_data(n, d, n_x)

true_te = lambda X: np.hstack([X[:, [0]]**2 + 1, np.ones((X.shape[0], n_x - 1))])
w = deepcopy(v)
v_test = v[:min(100, n)].copy()
v_test[:, 0] = np.linspace(np.percentile(v[:, 0], 1), np.percentile(v[:, 0], 99), min(100, n))

In [4]:
data = to_df(treatment=x, outcome=y, v=v)
test_data = to_df(v=v_test)

outcome = 'outcome'
treatment = 'treatment'
adjustment = data.columns[2:]

In [5]:
from sklearn.preprocessing import OneHotEncoder
# oh = OneHotEncoder()
# x = oh.fit_transform(x).toarray()

In [6]:
gt = _GrfTree()
gt._fit_with_array(x=x, y=y.squeeze(), w=w, v=v, i=1)

building 2-th tree


In [7]:
gt._predict_with_array(w, v)

array([ 0.31388521, -0.30602605,  1.71559724, ...,  1.34296921,
        0.4528414 ,  1.7961405 ])

In [10]:


grf = NaiveGrf(n_jobs=1, n_estimators=10)
grf.verbose = 1


In [11]:
grf.fit(
    data=data, outcome=outcome, treatment=treatment, adjustment=adjustment, covariate=adjustment
)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


building 1-th tree
building 2-th tree
building 3-th tree
building 4-th tree
building 5-th tree
building 6-th tree
building 7-th tree
building 8-th tree
building 9-th tree
building 10-th tree


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:   32.0s finished
